In [1]:
from datetime import date,timedelta
import requests
from bs4 import BeautifulSoup
import geograpy 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import openpyxl
import html2text
import pycountry
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
import pt_core_news_sm
file=openpyxl.Workbook()
std=file['Sheet']
std.append(['url','flag','Country','News'])
h = html2text.HTML2Text()
# Ignore converting links from HTML
h.ignore_links = True
today = date.today()
yesterday = today - timedelta(days=1)
print("Yesterday's date:", yesterday)
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found") 
keywords=['new pandemic','new epidemic','new outbreak','new lockdown','new quarantine','deaths','new viral outbreak']
article=[]
for query in keywords:  
    for j in search(query+" after:"+str(yesterday), tld="com", num=100, stop=5, pause=2): 
        if j.find('corona')==-1 and j.find('covid')==-1:
            print(j)
            article.append(j)


Yesterday's date: 2020-08-28
https://www.livemint.com/mint-lounge/features/a-virtual-new-beginning-for-baro-11598677030010.html
https://health.economictimes.indiatimes.com/news/industry/china-study-warns-of-possible-new-pandemic-virus-from-pigs/76721325
https://www.bbc.com/news/world-51235105
https://www.cdc.gov.tw/En
https://www.cdc.gov.tw/En/Bulletin/Detail/pevkg6JalT40I4uB6wAWcg?typeid=158
https://www.cdc.gov.tw/En/Bulletin/Detail/YS_6xZ01kBHnOpamUwyEgA?typeid=158
https://www.cdc.gov.tw/En/Bulletin/List/7tUXjTBf6paRvrhEl-mrPg
https://www.cdc.gov.tw/En/Category/Page/yZOu-4cGeu77HDyzE0ojqg
http://outbreaknewstoday.com/
https://www.bbc.com/news/world-51235105
https://apnews.com/VirusOutbreak
https://indianexpress.com/article/india/india-unlock-4-0-guidelines-6575245/
https://www.india.com/news/india/lockdown-in-uttar-pradesh-religious-social-programmes-restricted-in-state-till-sept-30-weekend-shutdown-to-be-strictly-implemented-4125020/
https://economictimes.indiatimes.com/topic/new-lo

In [2]:
a=[]
raspin=1
for it in article:
    try:
        #print(i)
        flag=0
        url = it#"https://health.economictimes.indiatimes.com/news/industry/china-study-warns-of-possible-new-pandemic-virus-from-pigs/76721325"
        places = geograpy.get_place_context(url=url) 
        Location=[]
        page=requests.get(url)
        soup = BeautifulSoup.get_text(BeautifulSoup(page.text, 'html.parser'))
        viral_data=soup.find("virus")
        virus=soup[viral_data-10:viral_data+5]
        #print(virus)
        places = geograpy.get_place_context(url=url) 
        max=0
        for i in places.country_mentions:
            try:
                if pycountry.countries.lookup(i[0]):
                    Location.append(pycountry.countries.lookup(i[0]).name)
                    if max< i[1]:
                        max=i[1]
                        max_location=i[0]
            except:
                pass
        #nlp = pt_core_news_sm.load()
        if soup.find('Denied')!=-1 or soup.find('connect to the server')!=-1 :
            print("Suka")
            continue
        nlp=spacy.load('en_core_web_sm')#en_core_web_sm.load()
        f=soup
        text=f
        #text = f.replace("  ","")
        #text=f.replace("\n"," ")
        doc = nlp(text)
        entities=[]
        something=[]
        something_significant=""
        matches = ["affect", "dea", "case","patients"]
        for i in doc.ents:
            if i.label_=='CARDINAL':
                entities.append((i.text,text[i.start_char-10:i.end_char+10], i.label_))#, i.label))
                something.append(text[i.start_char-30:i.end_char+30])
                if any(x in text[i.start_char:i.end_char+20] for x in matches):
                    #print(text[i.start_char:i.end_char+20])
                    #print(text[i.start_char:i.end_char+20].find("dea"))
                    #print(matches)
                    if text[i.start_char:i.end_char+20].find("affect")!=-1:
                        matches.remove("affect")
                    if text[i.start_char:i.end_char+20].find("dea")!=-1:
                        matches.remove("dea")
                    if text[i.start_char:i.end_char+20].find("case")!=-1:
                        matches.remove("case")
                    if text[i.start_char:i.end_char+20].find("patients")!=-1:
                        matches.remove("patients")
                    if matches==[]:
                        continue
                    else:
                        pass
                    '''
                    for num in range(i.start_char,i.start_char-100,-1):
                        if text[i.start_char-num]=="." or text[i.start_char-num-1:i.start_char-num]=="  " :
                            break
                    for num2 in range(i.end_char,i.end_char+100,1):
                        try:
                            if text[i.end_char+num2]==".":
                                break
                        except:
                            break
                    '''
                    something_significant=something_significant+text[i.start_char:i.end_char+35]+". "
                    #something_significant.append(text[i.start_char:i.end_char+100])
            else: 
                pass 
        corpus = [sent.text.lower() for sent in doc.sents ]
        cv = CountVectorizer(stop_words=list(STOP_WORDS))   
        cv_fit=cv.fit_transform(corpus)    
        word_list = cv.get_feature_names();    
        count_list = cv_fit.toarray().sum(axis=0)
        word_frequency = dict(zip(word_list,count_list))
        val=sorted(word_frequency.values())
        higher_word_frequencies = [word for word,freq in word_frequency.items() if freq in val[-3:]]
        #print("\nWords with higher frequencies: ", higher_word_frequencies)# gets relative frequency of words
        higher_frequency = val[-1]
        for word in word_frequency.keys():  
            word_frequency[word] = (word_frequency[word]/higher_frequency)
        sentence_rank={}
        for sent in doc.sents:
            for word in sent :       
                if word.text.lower() in word_frequency.keys():            
                    if sent in sentence_rank.keys():
                        sentence_rank[sent]+=word_frequency[word.text.lower()]
                    else:
                        sentence_rank[sent]=word_frequency[word.text.lower()]
        top_sentences=(sorted(sentence_rank.values())[::-1])
        top_sent=top_sentences[:3]
        summary=[]
        for sent,strength in sentence_rank.items():  
            if strength in top_sent:
                summary.append(sent)
            else:
                continue 
        #print(url,max_location,something_significant,sep=" ",end="\n################3\n")
        summary=""
        for i in summary:
            print(i,end=" ")
            summary=summary+" "+i
        #print("\nLOCATION####################\n")
        #print(Location,end="\nSomething####################\n")
        #print(something,end="\nEntities####################\n")
        #print(entities,end="\n###########################################################################3\n ")
        #strings=strings+"\n####################\n"+str(Location)+"\n####################\n"+str(entities)+"\n############"+str(something)+"###############################################################3\n "
        strings=url+"\n"+str(max_location)+"\n"+str(something_significant)+"\n##########\n"
        if len(something_significant)>1 and len(virus)!=0:
            with open("ReportFile.txt", "a") as file1: 
                file1.write(strings)
                std.append([url,1,str(max_location),str(max_location)+" "+virus+" "+str(something_significant)+" "+url,summary])
                raspin+=1
    except:
        pass
file.save("ReportFile.xlsx")            

rishCoronavirus
'pandemic virus




Coronavirus
 of Enterovirus
vel coronavirus
 of Enterovirus
 of Enterovirus
 of Enterovirus
West Nile virus




Coronavirus
worldwide virus
the coronavirus
ews
Coronavirus
vers from virus
vers from virus

Suka




Coronavirus
vers from virus
worldwide virus
new coronavirus
spread of virus
'pandemic virus
